#importing the libraries


In [5]:
import numpy as np
import pandas as pd

#Importing the dataset

In [6]:
rating_df = pd.read_csv('ratings.csv',usecols=['userId','movieId','rating'])
movie_df= pd.read_csv('movies.csv',usecols=['movieId','title'])

In [7]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [8]:
movie_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


#Splitting the dataset

In [9]:
from sklearn.model_selection import train_test_split
train_rating_df,test_rating_df = train_test_split(rating_df,test_size=0.2,random_state=1)

#Create an user-item interaction pivot table

In [10]:
data_table = rating_df.pivot_table(index='movieId',columns='userId' , values='rating')
data_table =data_table.fillna(0)
train_table = train_rating_df.pivot_table(index='movieId',columns='userId' , values='rating')
train_table =train_table.fillna(0)

In [11]:
data_table.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
train_table.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,0.0,2.5,0.0,0.0,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


#Create KNN model

In [13]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(n_neighbors=20,algorithm='brute',metric='cosine')

#Train KNN on the training set

In [14]:
model.fit(data_table)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

#Reccomendation system

#Input the user to recommend movies to

In [15]:
selected_id= input(' Enter the userId of the user to recommend to : ')
selected_id = int(selected_id)

 Enter the userId of the user to recommend to : 60


#Getting the selected user's rated movies

In [16]:
selected_user= data_table[selected_id]
selected_user = selected_user.loc[selected_user!=0]
selected_user.head()

movieId
48     3.0
50     3.0
60     3.0
318    4.0
362    4.0
Name: 60, dtype: float64

#List of movies that rated by at least 50 users

In [17]:
stats= rating_df.groupby('movieId').agg({'rating':[np.size]})
stats.head(10 )
stats= stats[stats['rating']['size']>=50]

In [18]:
stats.head()

,rating
,size
movieId,
1,215
2,110
3,52
6,102
7,54


#Function to get recommendations and predict ratings that user would give to each movie

In [19]:
def get_recommendation():
  similar_candidates_rating= pd.Series(dtype='float64')
  similar_candidates_score= pd.Series(dtype='float64')
  for i in range(0,len(selected_user.index)):
    similar=model.kneighbors([data_table.loc[selected_user.index[i]]],return_distance=True)
    sim_score=similar[0]
    sim_id=similar[1]
    sim_id=np.array(list(map(lambda x: data_table.iloc[x].name,sim_id[0])))
    similar=pd.Series(data=sim_score[0],index=sim_id)
    similar=similar[similar!=0]
    similar=similar[similar.index.isin(stats.index)]
    similar_candidates_score=pd.concat([similar_candidates_score,similar])
    similar=similar.map(lambda x: x*selected_user.values[i])
    similar_candidates_rating = pd.concat([similar_candidates_rating,similar])
  return similar_candidates_rating,similar_candidates_score

In [20]:
similar_candidates_rating,similar_candidates_score= get_recommendation()

#Factoring the recommendation score to the scale of 1-5

In [21]:
similar_candidates_rating.sort_values(inplace=True,ascending=False)
similar_candidates_score.sort_values(inplace=True,ascending=False)
filtered_candidates_rating_sum= similar_candidates_rating.groupby(similar_candidates_rating.index).sum()
filtered_candidates_score_sum= similar_candidates_score.groupby(similar_candidates_score.index).sum()

In [22]:
similar_movies=filtered_candidates_rating_sum.index
pred_rating= pd.Series(dtype='float64',index=similar_movies)
for i in range(0,len(similar_movies)):
  pred_rating[similar_movies[i]]= filtered_candidates_rating_sum[similar_movies[i]]/filtered_candidates_score_sum[similar_movies[i]]

#Making Recommendations to the selected user

#The user's orignal highest rated films

In [23]:
selected_user.sort_values(inplace=True,ascending=False)
selected_user_df = pd.DataFrame(selected_user).reset_index()
selected_user_df.columns = ['movieId', 'predicted_rating']
selected_user_df = pd.merge(selected_user_df,movie_df)
selected_user_df.head(10)

,movieId,predicted_rating,title
0,58559,5.0,"Dark Knight, The (2008)"
1,527,5.0,Schindler's List (1993)
2,858,5.0,"Godfather, The (1972)"
3,2150,4.0,"Gods Must Be Crazy, The (1980)"
4,6016,4.0,City of God (Cidade de Deus) (2002)
5,318,4.0,"Shawshank Redemption, The (1994)"
6,362,4.0,"Jungle Book, The (1994)"
7,3424,4.0,Do the Right Thing (1989)
8,783,4.0,"Hunchback of Notre Dame, The (1996)"
9,805,4.0,"Time to Kill, A (1996)"


#Recommendations made by KNN

In [24]:
print("List of recommendations for user {}".format(selected_id))
pred_rating.sort_values(inplace=True,ascending=False)
pred_rating_df = pd.DataFrame(pred_rating).reset_index()
pred_rating_df.columns = ['movieId', 'predicted_rating']
final_pred_df = pd.merge(pred_rating_df,movie_df)
final_pred_df.head(10)

List of recommendations for user 60


,movieId,predicted_rating,title
0,2028,5.0,Saving Private Ryan (1998)
1,1210,5.0,Star Wars: Episode VI - Return of the Jedi (1983)
2,6874,5.0,Kill Bill: Vol. 1 (2003)
3,1198,5.0,Raiders of the Lost Ark (Indiana Jones and the...
4,1208,5.0,Apocalypse Now (1979)
5,2762,5.0,"Sixth Sense, The (1999)"
6,3147,5.0,"Green Mile, The (1999)"
7,6539,5.0,Pirates of the Caribbean: The Curse of the Bla...
8,33794,5.0,Batman Begins (2005)
9,377,5.0,Speed (1994)
